<html>
<font size = 6 color = Red align = center>
Principal Component Analysis - Explained with Wine dataset<br/>
</font>
</html>

<html>
<font size = 4 color = blue>
<u> Author: </u> <br />
</font>
<font size = 4 color = green>
Sulekha Aloorravi <br/>
</font>
</html>

<html>
<font color = Red>
<b>Objective:</b> <br />
</font>
<font color = DarkBlue>
Dimensionality Reduction through Principal Component Analysis on the Wine data set.
</font>
</html>

<html>
<font color = Red>
<b>Data Set:</b> <br />
</font>
<font color = Blue>
https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data
</font>
</html>

<html>
<font color = Red>
<b>Data details:</b> <br />
</font>
<font color = Blue>
https://archive.ics.uci.edu/ml/machine-learning-databases/wine/ <br />
</font>
<font color = DarkBlue>
The attributes are <br />
1) Alcohol<br />
2) Malic acid<br />
3) Ash<br />
4) Alcalinity of ash  <br />
5) Magnesium<br />
6) Total phenols<br />
7) Flavanoids<br />
8) Non flavanoid phenols<br />
9) Proanthocyanins<br />
10) Color intensity<br />
11) Hue<br />
12) OD280/OD315 of diluted wines<br />
13) Proline   <br />
</font>
</html>

<html>
<font color = DarkBlue>
Let us load the data set from uci website.
</font>
</html>

In [1]:
#Basic python libraries
import pandas as pd
import numpy as np

In [2]:
#Plotly libraries used in this Project
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
init_notebook_mode(connected=True) #Plotly offline
from plotly.graph_objs import *
from plotly import figure_factory

In [3]:
colnames = ["Class","Alcohol","Malicacid","Ash","Alcalinityofash","Magnesium","Totalphenols","Flavanoids","Nonflavanoidphenols","Proanthocyanins","Colorintensity","Hue","Dilutedwines","Proline"]
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', names= colnames)


In [4]:
df_wine.head(5)

,Class,Alcohol,Malicacid,Ash,Alcalinityofash,Magnesium,Totalphenols,Flavanoids,Nonflavanoidphenols,Proanthocyanins,Colorintensity,Hue,Dilutedwines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


<html>
<font color = DarkBlue>
Let us do some Exploratory Analysis before getting into PCA by: <br />
Finding variables with Strong Correlation between each other. <br />
</font>
</html>

In [5]:
# Extract values and row, column names
df_corr = df_wine.corr()
arr = df_corr.values
index_names = df_corr.index
col_names = df_corr.columns

#  Get indices where such threshold is crossed; avoid diagonal elems
R,C = np.where(np.triu(arr,1)>0.5)

# Arrange those in columns and put out as a dataframe
out_arr = np.column_stack((index_names[R],col_names[C],arr[R,C]))
df_out = pd.DataFrame(out_arr,columns=[['row_name','col_name','value']])

df_out

,row_name,col_name,value
0,Class,Alcalinityofash,0.517859
1,Alcohol,Colorintensity,0.546364
2,Alcohol,Proline,0.64372
3,Totalphenols,Flavanoids,0.864564
4,Totalphenols,Proanthocyanins,0.612413
5,Totalphenols,Dilutedwines,0.699949
6,Flavanoids,Proanthocyanins,0.652692
7,Flavanoids,Hue,0.543479
8,Flavanoids,Dilutedwines,0.787194
9,Proanthocyanins,Dilutedwines,0.519067


<html>
<font color = DarkBlue>
Ignoring the Class variable, there are 10 pairs of features with correlations of >0.50 out of total 13 unique features. <br />
Now, let us apply Principal Component Analysis on the wine data set.

</font>
</html>

<html>
<font color = Red>
<b>Concept:</b> <br />
</font>
<font color = DarkBlue>
Principal component Analysis is an algorithm which would help in visualizing, compressing the data and finding a representation that is more informative for further processing. <br />
PCA is a method that rotates the dataset in a way such that the rotated features are statistically uncorrelated. <br />
This rotation is often followed by selecting only a subset of the new features, according to how important they are for explaining the data. <br />
It is often useful to measure data in terms of its principal components rather than on a normal x-y axis. <br />
Principal components are the underlying structure in the data. <br />
They are the directions where there is the most variance, the directions where the data is most spread out. <br /><br />
The wine dataset we have uploaded here will be used to understand this concept of PCA.
</font>
</html>

<html>
<font color = Red>
<b>Steps:</b> <br />
</font>
<font color = DarkBlue>
PCA involves following broad level steps – <br />
1. Standardize the d-dimensional dataset.<br />
2. Construct the covariance matrix.<br />
3. Decompose the covariance matrix into its eigenvectors and eigenvalues.<br />
4. Select k eigenvectors that correspond to the k largest eigenvalues, where k is the dimensionality of the new feature subspace ( k≤d ).<br />
5. Construct a projection matrix W from the "top" k eigenvectors.<br />
6. Transform the d-dimensional input dataset x using the projection matrix W to obtain the new k-dimensional feature subspace
</font>
</html>

<html>
<font color = Purple Size = 4>
<b>1. Standardize the d-dimensional dataset.</b> <br />
</font>
</html>


<html>
<font color = DarkBlue>
Remove Target Variable column "Class" from the data set and store the rest of the independent columns in an array <b>X</b>

</font>
</html>

In [6]:
X = df_wine.iloc[:, 1:].values
Y = df_wine.iloc[:, 0].values

<html>
<font color = Red>
<b>Standard Scaler</b> <br />
</font>
<font color = DarkBlue>
The StandardScaler in scikit-learn ensures that for each feature, the mean is zero, and the variance is one, bringing all features to the same magnitude. <br />
However, this scaling does not ensure any particular minimum and maximum values for the features.<br />
</font>
</html>

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_std = sc.fit_transform(X)
X_std

array([[ 1.51861254, -0.5622498 ,  0.23205254, ...,  0.36217728,
         1.84791957,  1.01300893],
       [ 0.24628963, -0.49941338, -0.82799632, ...,  0.40605066,
         1.1134493 ,  0.96524152],
       [ 0.19687903,  0.02123125,  1.10933436, ...,  0.31830389,
         0.78858745,  1.39514818],
       ..., 
       [ 0.33275817,  1.74474449, -0.38935541, ..., -1.61212515,
        -1.48544548,  0.28057537],
       [ 0.20923168,  0.22769377,  0.01273209, ..., -1.56825176,
        -1.40069891,  0.29649784],
       [ 1.39508604,  1.58316512,  1.36520822, ..., -1.52437837,
        -1.42894777, -0.59516041]])

<html>
<font color = Purple Size = 4>
<b>2. Construct the covariance matrix.</b> <br />
</font>
</html>


<html>
<font color = DarkBlue>
The classic approach to PCA is to perform the eigen decomposition on the covariance matrix Σ, which is a d×d matrix where each element represents the covariance between two features. <br />
</font>
</html>

In [8]:
import numpy as np
cov_mat = np.cov(X_std.T)
print('Covariance Matrix \n%s', cov_mat)

Covariance Matrix 
%s [[ 1.00564972  0.09493026  0.21273976 -0.31198788  0.27232816  0.29073446
   0.23815287 -0.15681042  0.13747022  0.549451   -0.07215255  0.07275191
   0.64735687]
 [ 0.09493026  1.00564972  0.16497228  0.29013035 -0.05488343 -0.3370606
  -0.41332866  0.29463237 -0.22199334  0.25039204 -0.56446685 -0.37079354
  -0.19309537]
 [ 0.21273976  0.16497228  1.00564972  0.44587209  0.28820583  0.12970824
   0.11572743  0.1872826   0.00970647  0.2603499  -0.07508874  0.00393333
   0.22488969]
 [-0.31198788  0.29013035  0.44587209  1.00564972 -0.0838039  -0.32292752
  -0.353355    0.36396647 -0.19844168  0.01883781 -0.27550299 -0.27833221
  -0.44308618]
 [ 0.27232816 -0.05488343  0.28820583 -0.0838039   1.00564972  0.21561254
   0.19688989 -0.25774204  0.23777643  0.20107967  0.05571118  0.06637684
   0.39557317]
 [ 0.29073446 -0.3370606   0.12970824 -0.32292752  0.21561254  1.00564972
   0.86944804 -0.45247731  0.61587304 -0.05544792  0.43613151  0.70390388
   0.50092909]
 

<html>
<font color = Purple Size = 4>
<b>3. Decompose the covariance matrix into its eigenvectors and eigenvalues.</b> <br />
</font>
</html>

<html>
<font color = DarkBlue>
Eigendecomposition is the factorization of a matrix into a canonical form, whereby the matrix is represented in terms of its eigenvalues and eigenvectors. Only diagonalizable matrices can be factorized in this way. <br />
When we get a set of data points, we can deconstruct the set into eigenvectors and eigenvalues. <br />
Eigenvectors and values exist in pairs: every eigenvector has a corresponding eigenvalue. <br />
An eigenvector is a direction and an eigenvalue is a number, telling you how much variance is there in the data in that direction
</font>
</html>

In [9]:
eigen_vals, eigen_vecs = np.linalg.eig(cov_mat)
print('Eigen Vectors \n%s', eigen_vecs)

Eigen Vectors 
%s [[-0.1443294  -0.48365155  0.20738262 -0.0178563   0.26566365 -0.21353865
  -0.05639636  0.39613926  0.01496997  0.26628645 -0.50861912 -0.22591696
   0.21160473]
 [ 0.24518758 -0.22493093 -0.08901289  0.53689028 -0.03521363 -0.53681385
   0.42052391  0.06582674  0.02596375 -0.12169604  0.07528304  0.07648554
  -0.30907994]
 [ 0.00205106 -0.31606881 -0.6262239  -0.21417556  0.14302547 -0.15447466
  -0.14917061 -0.17026002 -0.14121803  0.04962237  0.30769445 -0.49869142
  -0.02712539]
 [ 0.23932041  0.0105905  -0.61208035  0.06085941 -0.06610294  0.10082451
  -0.28696914  0.42797018  0.09168285  0.05574287 -0.20044931  0.47931378
   0.05279942]
 [-0.14199204 -0.299634   -0.13075693 -0.35179658 -0.72704851 -0.03814394
   0.3228833  -0.15636143  0.05677422 -0.06222011 -0.27140257  0.07128891
   0.06787022]
 [-0.39466085 -0.06503951 -0.14617896  0.19806835  0.14931841  0.0841223
  -0.02792498 -0.40593409 -0.46390791  0.30388245 -0.28603452  0.30434119
  -0.32013135]
 [-0.

In [10]:
print('\nEigenvalues \n%s' % eigen_vals)


Eigenvalues 
[ 4.73243698  2.51108093  1.45424187  0.92416587  0.85804868  0.64528221
  0.55414147  0.35046627  0.10396199  0.16972374  0.29051203  0.22706428
  0.25232001]


<html>
<font color = Purple Size = 4>
<b>4. Select k eigenvectors that correspond to the k largest eigenvalues, where k is the dimensionality of the new feature subspace ( k≤d ).</b> <br />
</font>
</html>

<html>
<font color = DarkBlue>
The typical goal of a PCA is to reduce the dimensionality of the original feature space by projecting it onto a smaller subspace, where the eigenvectors will form the axes.  <br />
However, the eigenvectors only define the directions of the new axis, since they have all the same unit length 1. <br />
</font>
</html>

In [11]:
for ev in eigen_vecs:
    np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))
print('All values have the same unit length of 1')

All values have the same unit length of 1


<html>
<font color = DarkBlue>
Explained Variance is a measure which can be calculated from the eigen values and it will tell us the quantity of information or variance that can be attributed to each principal components. <br />
Based on this, we can decide on the number of principal components we are going to choose for our new feature subspace.
</font>
</html>

In [12]:
tot = sum(eigen_vals)
var_exp = [(i / tot)*100 for i in sorted(eigen_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

In [13]:
trace1 = Bar(
        x=['PC %s' %i for i in range(1,14)],
        y=var_exp,
        showlegend=True,
        name='Individual explained variance')

trace2 = Scatter(
        x=['PC %s' %i for i in range(1,14)], 
        y=cum_var_exp,
        name='Cumulative explained variance')

data = Data([trace1, trace2])

layout=Layout(
        yaxis=YAxis(title='Explained variance in percent'),
        title='Explained variance by Principal components')

fig = Figure(data=data, layout=layout)
iplot(fig)

<html>
<font color = DarkBlue>
The above graph clearly shows that around 92% of variance can be explained by first eight principal components which also means that first eight principal components contains 92% of the information.
</font>
</html>

<html>
<font color = Purple Size = 4>
<b>5. Construct a projection matrix W from the "top" k eigenvectors.</b> <br />
</font>
</html>

<html>
<font color = DarkBlue>
Now, let us construct the Projection matrix that will be used to transform the wine dataset into the new feature subspace. <br />
This Projection matrix is a matrix of concatenated top k eigenvectors.
</font>
</html>

In [14]:
# Make a list of (eigenvalue, eigenvector) tuples
eigen_pairs = [(np.abs(eigen_vals[i]), eigen_vecs[:,i]) for i in range(len(eigen_vals))]

In [15]:
# Sort the (eigenvalue, eigenvector) tuples from high to low
eigen_pairs.sort()
eigen_pairs.reverse()

In [16]:
# Confirm that the list is correctly sorted by decreasing eigenvalues
print('Eigenvalues in descending order:')
for i in eigen_pairs:
    print(i[0])

Eigenvalues in descending order:
4.73243697758
2.51108092965
1.45424186785
0.924165866825
0.858048676537
0.645282212468
0.554141466246
0.350466274946
0.290512032694
0.252320010361
0.227064281731
0.16972373898
0.103961991821


In [17]:
w = np.hstack((eigen_pairs[0][1][:, np.newaxis],
               eigen_pairs[1][1][:, np.newaxis],
               eigen_pairs[2][1][:, np.newaxis],
               eigen_pairs[3][1][:, np.newaxis],
               eigen_pairs[4][1][:, np.newaxis],
               eigen_pairs[5][1][:, np.newaxis],
               eigen_pairs[6][1][:, np.newaxis],
               eigen_pairs[7][1][:, np.newaxis]))
print('Matrix W:\n', w)

Matrix W:
 [[-0.1443294  -0.48365155  0.20738262 -0.0178563   0.26566365 -0.21353865
  -0.05639636  0.39613926]
 [ 0.24518758 -0.22493093 -0.08901289  0.53689028 -0.03521363 -0.53681385
   0.42052391  0.06582674]
 [ 0.00205106 -0.31606881 -0.6262239  -0.21417556  0.14302547 -0.15447466
  -0.14917061 -0.17026002]
 [ 0.23932041  0.0105905  -0.61208035  0.06085941 -0.06610294  0.10082451
  -0.28696914  0.42797018]
 [-0.14199204 -0.299634   -0.13075693 -0.35179658 -0.72704851 -0.03814394
   0.3228833  -0.15636143]
 [-0.39466085 -0.06503951 -0.14617896  0.19806835  0.14931841  0.0841223
  -0.02792498 -0.40593409]
 [-0.4229343   0.00335981 -0.1506819   0.15229479  0.10902584  0.01892002
  -0.06068521 -0.18724536]
 [ 0.2985331  -0.02877949 -0.17036816 -0.20330102  0.50070298  0.25859401
   0.59544729 -0.23328465]
 [-0.31342949 -0.03930172 -0.14945431  0.39905653 -0.13685982  0.53379539
   0.37213935  0.36822675]
 [ 0.0886167  -0.52999567  0.13730621  0.06592568  0.07643678  0.41864414
  -0.22

<html>
<font color = DarkBlue>
By executing the above code, we have created a 13 x 8-dimensional projection matrix W from the top 8 eigenvectors. <br />
This Projection matrix is a matrix of concatenated top k eigenvectors.
</font>
</html>

<html>
<font color = Purple Size = 4>
<b>6. Transform the d-dimensional input dataset x using the projection matrix W to obtain the new k-dimensional feature subspace.</b> <br />
</font>
</html>

<html>
<font color = DarkBlue>
Using the projection matrix, we can now transform each dimension into the PCA subspace by obtaining new 8-dimensional sample space consisting of 8 new features.
</font>
</html>

In [18]:
X_pca = X_std.dot(w)

In [19]:
X_pca

array([[-3.31675081, -1.44346263,  0.16573904, ..., -0.22388013,
         0.59642655, -0.06513909],
       [-2.20946492,  0.33339289,  2.02645737, ..., -0.92712024,
         0.05377561, -1.02441595],
       [-2.51674015, -1.0311513 , -0.98281867, ...,  0.54927605,
         0.42420545,  0.34421613],
       ..., 
       [ 2.67783946, -2.76089913,  0.94094188, ...,  0.27306792,
         0.67923541, -0.0470238 ],
       [ 2.38701709, -2.29734668,  0.5506962 , ...,  1.17878298,
         0.63397527, -0.39082877],
       [ 3.20875816, -2.76891957, -1.01391366, ...,  0.29609218,
         0.00574126,  0.29291373]])

<html>
<font color = DarkBlue>
New 8 dimensional matrix converted into a dataframe will be as follows:
</font>
</html>

In [20]:
pca_df = pd.DataFrame(X_pca, columns = [["PC1","PC2","PC3","PC4","PC5","PC6","PC7","PC8"]])
pca_df.head(10)

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8
0,-3.316751,-1.443463,0.165739,-0.215631,-0.693043,-0.223880,0.596427,-0.065139
1,-2.209465,0.333393,2.026457,-0.291358,0.257655,-0.927120,0.053776,-1.024416
2,-2.516740,-1.031151,-0.982819,0.724902,0.251033,0.549276,0.424205,0.344216
3,-3.757066,-2.756372,0.176192,0.567983,0.311842,0.114431,-0.383337,-0.643593
4,-1.008908,-0.869831,-2.026688,-0.409766,-0.298458,-0.406520,0.444074,-0.416700
5,-3.050254,-2.122401,0.629396,-0.515637,0.632019,0.123431,0.401654,-0.394893
6,-2.449090,-1.174850,0.977095,-0.065831,1.027762,-0.620121,0.052891,0.371934
7,-2.059437,-1.608963,-0.146282,-1.192608,-0.076903,-1.439806,0.032376,-0.232979
8,-2.510874,-0.918071,1.770969,0.056270,0.892257,-0.129181,0.125285,0.499578
9,-2.753628,-0.789438,0.984247,0.349382,0.468553,0.163392,-0.874352,-0.150580


<html>
<font color = DarkBlue>
To get a better understanding of the above Principal components, let us plot the first 3 PCA dimensions that contribute to 66.5% of the information and have a look at them in three dimensional view.
</font>
</html>

In [21]:
x, y, z = pca_df["PC1"],pca_df["PC2"],pca_df["PC3"]

trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=6,
        color=Y,                
        colorscale='Viridis',   
        opacity=0.8
    )
)

data = [trace1]
layout = go.Layout( scene = dict(
                    xaxis = dict(
                         backgroundcolor="rgb(200, 200, 230)",
                         gridcolor="rgb(255, 255, 255)",
                         showbackground=True,
                         zerolinecolor="rgb(255, 255, 255)",
                        title='PCA 1'),
                    yaxis = dict(
                        backgroundcolor="rgb(230, 200,230)",
                        gridcolor="rgb(255, 255, 255)",
                        showbackground=True,
                        title='PCA 2'),
                    zaxis = dict(
                        backgroundcolor="rgb(230, 230,200)",
                        gridcolor="rgb(255, 255, 255)",
                        showbackground=True,
                        zerolinecolor="rgb(255, 255, 255)",
                        title='PCA 3'),),
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

<html>
<font color = DarkBlue>
Let us now check if there are any correlations between the new features. 
</font>
</html>

In [22]:
from plotly import figure_factory
fig = figure_factory.create_scatterplotmatrix(pca_df, diag='histogram', height=800, width=800)
iplot(fig)

In [23]:
# Extract values and row, column names
df_corr = pca_df.corr()
arr = df_corr.values
index_names = df_corr.index
col_names = df_corr.columns

#  Get indices where such threshold is crossed; avoid diagonal elems
R,C = np.where(np.triu(arr,1)>0.1)

# Arrange those in columns and put out as a dataframe
out_arr = np.column_stack((index_names[R],col_names[C],arr[R,C]))
df_out = pd.DataFrame(out_arr,columns=[['row_name','col_name','value']])

df_out

,row_name,col_name,value


<html>
<font color = DarkBlue>
There are no correlations between the new features. As stated in the beginning, PCA is a method that rotates the dataset in a way such that the rotated features are statistically uncorrelated. <br /><br />
<b>With the above steps and explanation, we have performed Dimensionality Reduction on the Wine data set through Principal Component Analysis.</b>
</font>
</html>